<a href="https://colab.research.google.com/github/Vikvas6/NN/blob/master/Lesson_3_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.2.0-rc2


In [0]:
(train_data, test_data), info = tfds.load(
    # Use the version pre-encoded with an ~8k vocabulary.
    'imdb_reviews/subwords8k', 
    # Return the train/test datasets as a tuple.
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    # Return (example, label) pairs from the dataset (instead of a dictionary).
    as_supervised=True,
    # Also return the `info` structure. 
    with_info=True)

Данные изначально представляют из себя набор отзыв - его бинарная оценка (положительный или отрцательный), при этом отзыв написан на человеческом языке, и его сначала надо токенизировать. В imdb_reviews есть несколько датасетов, и датасеты subwords8k и subwords32k содержат в себе уже токенизированные данные (с разным объёмом словаря). Я нашёл в документации разбор датасета subwords8k, поэтому взял его.

In [45]:
tf.data.experimental.cardinality(train_data)

<tf.Tensor: shape=(), dtype=int64, numpy=25000>

In [46]:
for data_example, label_example in train_data.take(1):
    print(data_example)
    print(label_example)

tf.Tensor(
[  62   18   41  604  927   65    3  644 7968   21   35 5096   36   11
   43 2948 5240  102   50  681 7862 1244    3 3266   29  122  640    2
   26   14  279  438   35   79  349  384   11 1991    3  492   79  122
  188  117   33 4047 4531   14   65 7968    8 1819 3947    3   62   27
    9   41  577 5044 2629 2552 7193 7961 3642    3   19  107 3903  225
   85  198   72    1 1512  738 2347  102 6245    8   85  308   79 6936
 7961   23 4981 8044    3 6429 7961 1141 1335 1848 4848   55 3601 4217
 8050    2    5   59 3831 1484 8040 7974  174 5773   22 5240  102   18
  247   26    4 3903 1612 3902  291   11    4   27   13   18 4092 4008
 7961    6  119  213 2774    3   12  258 2306   13   91   29  171   52
  229    2 1245 5790  995 7968    8   52 2948 5240 8039 7968    8   74
 1249    3   12  117 2438 1369  192   39 7975], shape=(163,), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)


In [47]:
tf.data.experimental.cardinality(test_data)

<tf.Tensor: shape=(), dtype=int64, numpy=25000>

Из датасета мы взяли так же метаинформацию о нём, которая в том числе содержит словарь, который использовался для токенизации датасета.

In [48]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(None,), dtype=tf.int64, encoder=<SubwordTextEncoder vocab_size=8185>),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Pot

In [0]:
encoder = info.features['text'].encoder

In [50]:
print ('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


Разобьём данные на батчи, причём используем функцию padded_batch, благодаря чему в каждом батче длина отзывов будет одинаковая - в человеческом языке предложения имеют разную длину, что неудобно при обучении модели, поэтому данные лучше выравнить, добивая каким-нибудь нулевым символом размер предложения до выбранного значения. Вообще padded_batch пока не решит нашу проблему, т.к. эта функция ровняет только батч, в дальнейшем мы воспользуемся специальным слоем в сети.

In [0]:
BUFFER_SIZE = 1000
train_batches = (
    train_data
    .shuffle(BUFFER_SIZE)
    .padded_batch(32))

test_batches = (
    test_data
    .padded_batch(32))

In [52]:
for example_batch, label_batch in train_batches.take(2):
  print("Batch shape:", example_batch.shape)
  print("label shape:", label_batch.shape)

Batch shape: (32, 805)
label shape: (32,)
Batch shape: (32, 768)
label shape: (32,)


In [53]:
model = keras.Sequential([
  keras.layers.Embedding(encoder.vocab_size, 16),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(64),
  keras.layers.Dense(1)])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                1088      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 132,113
Trainable params: 132,113
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [55]:
model.fit(train_batches,
            epochs=10,
            validation_data=test_batches,
            validation_steps=30)

Epoch 1/10
782/782 [==============================] - 8s 11ms/step - loss: 0.5506 - accuracy: 0.6581 - val_loss: 0.4262 - val_accuracy: 0.8448
Epoch 2/10
782/782 [==============================] - 9s 11ms/step - loss: 0.2959 - accuracy: 0.8764 - val_loss: 0.3537 - val_accuracy: 0.8656
Epoch 3/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2335 - accuracy: 0.9065 - val_loss: 0.4131 - val_accuracy: 0.8146
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 0.1998 - accuracy: 0.9202 - val_loss: 0.3810 - val_accuracy: 0.8594
Epoch 5/10
782/782 [==============================] - 8s 10ms/step - loss: 0.1793 - accuracy: 0.9286 - val_loss: 0.4725 - val_accuracy: 0.8552
Epoch 6/10
782/782 [==============================] - 8s 10ms/step - loss: 0.1577 - accuracy: 0.9391 - val_loss: 0.4299 - val_accuracy: 0.8635
Epoch 7/10
782/782 [==============================] - 8s 10ms/step - loss: 0.1442 - accuracy: 0.9442 - val_loss: 0.4658 - val_accuracy: 0.8573

In [56]:
loss, accuracy = model.evaluate(test_batches)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 [==============================] - 4s 6ms/step - loss: 0.5605 - accuracy: 0.8606
Loss:  0.5604603290557861
Accuracy:  0.8605999946594238
